In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras.callbacks import CSVLogger
import os
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
import datetime
tf.enable_eager_execution()

In [0]:
tf.keras.backend.clear_session()

In [0]:
sys.path.append('/content/drive/My Drive/convAE_2_classifier')
from dataset import Dataset

In [0]:
dataset = Dataset('cats_vs_dogs')
IMG_SIZE = 256
input_shape = (IMG_SIZE, IMG_SIZE, 3)
dataset.set_img_size(IMG_SIZE)
train, val, test = dataset.format_data()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incomplete79WJWJ/cats_vs_dogs-train.tfrecord
Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = val.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [2]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               #weights='imagenet')
                                               weights=None)
base_model.load_weights('imagenet')
base_model.trainable = False

NameError: ignored

In [0]:
for image_batch, label_batch in train_batches.take(1):
   pass

In [0]:
feature_batch = base_model(image_batch)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
prediction_layer = tf.keras.layers.Dense(1)

In [0]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [0]:
base_learning_rate = 0.0001
optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate)
model.compile(optimizer= optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
file_name = "mobileNet256"
dateTime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_path = "./drive/My Drive/convAE_2_classifier/logs/" + file_name +  "_model_history_" + dateTime + ".csv"
csv_logger = CSVLogger(csv_path, separator=',', append=True)
# log_dir="./drive/My Drive/convAE_2_classifier/logs/mobileNet_256X256_logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback  = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='epoch', histogram_freq=1, write_graph=True, write_images=True)
callbacks =[csv_logger]

In [0]:
initial_epochs = 10
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches, callbacks=callbacks)

Epoch 1/10
582/582 [==============================] - 69s 119ms/step - loss: 0.3484 - acc: 0.8271 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/10
582/582 [==============================] - 61s 105ms/step - loss: 0.1728 - acc: 0.9393 - val_loss: 0.2099 - val_acc: 0.8740
Epoch 3/10
582/582 [==============================] - 62s 106ms/step - loss: 0.1355 - acc: 0.9506 - val_loss: 0.1966 - val_acc: 0.8895
Epoch 4/10
582/582 [==============================] - 61s 105ms/step - loss: 0.1192 - acc: 0.9558 - val_loss: 0.1896 - val_acc: 0.8968
Epoch 5/10
582/582 [==============================] - 61s 104ms/step - loss: 0.1096 - acc: 0.9583 - val_loss: 0.1851 - val_acc: 0.9033
Epoch 6/10
582/582 [==============================] - 61s 104ms/step - loss: 0.1032 - acc: 0.9606 - val_loss: 0.1820 - val_acc: 0.9071
Epoch 7/10
582/582 [==============================] - 61s 105ms/step - loss: 0.0985 - acc: 0.9619 - val_loss: 0.1797 - val_acc: 0.9106
Epoch 8/10
582/582 [===========================

In [0]:
base_model.trainable = True

In [0]:
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [0]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [0]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches, callbacks=callbacks)

Epoch 10/20
582/582 [==============================] - 70s 120ms/step - loss: 0.0176 - acc: 0.9951 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 11/20
582/582 [==============================] - 68s 116ms/step - loss: 0.0083 - acc: 0.9983 - val_loss: 0.0794 - val_acc: 0.9746
Epoch 12/20
582/582 [==============================] - 67s 116ms/step - loss: 0.0037 - acc: 0.9995 - val_loss: 0.0878 - val_acc: 0.9759
Epoch 13/20
582/582 [==============================] - 68s 116ms/step - loss: 0.0018 - acc: 0.9997 - val_loss: 0.1003 - val_acc: 0.9785
Epoch 14/20
582/582 [==============================] - 68s 116ms/step - loss: 0.0012 - acc: 0.9998 - val_loss: 0.1163 - val_acc: 0.9789
Epoch 15/20
582/582 [==============================] - 68s 117ms/step - loss: 8.8289e-04 - acc: 0.9998 - val_loss: 0.1399 - val_acc: 0.9781
Epoch 16/20
582/582 [==============================] - 68s 116ms/step - loss: 6.3764e-04 - acc: 0.9999 - val_loss: 0.1540 - val_acc: 0.9794
Epoch 17/20
582/582 [===========

In [0]:
model.save("/content/drive/My Drive/convAE_2_classifier/weights/mobileNetV2_finetuned_256X256_model.h5")